In [1]:
from utils.libs import *

In [2]:
# --- Read config file for runnig notebook.

conf_file_path = 'conf.txt'
conf_data = read_conf_file(conf_file_path)

In [3]:
typename = 'QueryConstraints'
field_names = "image;date;timestamp;hidden_features;image_size;status,data_downloaded".split(";")
fields_type = 'str,str,str,int,str,str,str'.split(',')
QueryConstraints = collections.namedtuple(typename, field_names)

def map_func(field_type):
    return dict(zip(['type', 'val'], [eval(field_type), None]))
fields_list = list(map(map_func, fields_type))

constraints = QueryConstraints._make(fields_list)

In [4]:
image = dict(zip(['type', 'val'], [str, list(sorted("".split(";")))]))
date = dict(zip(['type', 'val'], [str, list(sorted("".split(";")))]))
timestamp = dict(zip(['type', 'val'], [str, list(sorted("".split(";")))]))
hidden_features = dict(zip(['type', 'val'], [int, list(sorted("".split(";")))]))
image_size = dict(zip(['type', 'val'], [str, list(sorted("".split(";")))]))
status = dict(zip(['type', 'val'], [str, list(sorted("done".split(";")))]))
data_downloaded = dict(zip(['type', 'val'], [str, list(sorted("TRUE".split(";")))]))

fields_list = [image, date, timestamp, hidden_features, image_size, status, data_downloaded]
constraints = QueryConstraints._make(fields_list)

pprint(constraints)

TypeError: Expected 6 arguments, got 7

In [ ]:
# --- Fetch data

records_list, result_dict_df, query_str, chained_constraints = fetch_data_by_constraints(
    conf_data, constraints, fetch_data_downloaded = False)

In [ ]:
data = list(map(operator.methodcaller('_asdict'), records_list))
records_df = pd.DataFrame(data = data)

records_df.head(5)

In [ ]:
print('full path:', records_df['full_path'].values[0])
print('base name:', os.path.basename(records_df['full_path'].values[0]))

base_filename = os.path.basename(records_df['full_path'].values[0])
val = str(records_df['full_path'].values[0])
val.replace(f'{base_filename}', 'train.log')

In [ ]:
def update_full_path(old_full_path, target_filename = 'train.log'):
    base_filename = os.path.basename(old_full_path)
    updated_full_path = old_full_path.replace(f'{base_filename}', f'{target_filename}')
    return updated_full_path
records_df['train_log'] = list(map(update_full_path, records_df['full_path'].values))
def update_full_path(old_full_path, target_filename = 'parser_logged.txt'):
    base_filename = os.path.basename(old_full_path)
    updated_full_path = old_full_path.replace(f'{base_filename}', f'{target_filename}')
    return updated_full_path
records_df['parser_logged'] = list(map(update_full_path, records_df['full_path'].values))

In [ ]:
records_df.head(5)

In [ ]:
records_df['train_log'].values[0:5]

In [ ]:
records_df['parser_logged'].values[0:5]

In [ ]:
original_images_dict = {'cameramen': {'width': 512, 'heigth': 512}}

In [ ]:
trd, failed_read = get_info_from_logged_parser(
    parser_logged_files = records_df['parser_logged'][:],
    train_log_files = records_df['train_log'].values[:],
    original_images_dict = original_images_dict,
    gpu_mode = True)

In [ ]:
failed_read

In [ ]:
data = trd.get_all_records_processed_for_query_insert()

In [ ]:
table_name = 'table_runs_details'
insert_data_read_from_logs(conf_data, table_name, trd.get_all_records())